In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import os
import json

In [2]:
save_path = os.getcwd()

In [4]:
mlist = pd.read_csv(os.path.join(save_path, "movie_list.csv"), encoding="euc-kr")

In [7]:
mlist

,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companyCd,companyNm
0,20191029,82년생 김지영,"KIM JI-YOUNG, BORN 1982",2019,20191023,장편,개봉,한국,드라마,한국,드라마,김도영,20192181.0,(주)봄바람영화사
1,20191589,말레피센트 2,Maleficent: Mistress of Evil,2019,20191017,장편,개봉,미국,"어드벤처,가족,판타지",미국,어드벤처,요아킴 뢰닝,NaN,NaN
2,20199950,조커,Joker,2019,20191002,장편,개봉,미국,"액션,드라마,스릴러",미국,액션,토드 필립스,NaN,NaN
3,20195920,가장 보통의 연애,Crazy Romance,2019,20191002,장편,개봉,한국,"멜로/로맨스,코미디",한국,멜로/로맨스,김한결,20100450.0,영화사 집
4,20196136,람보 : 라스트 워,Rambo: Last Blood,2019,20191023,장편,개봉,미국,액션,미국,액션,애드리언 그런버그,NaN,NaN
5,20184483,퍼펙트맨,Man of Men,2019,20191002,장편,개봉,한국,"코미디,드라마",한국,코미디,용수,20187561.0,(주)맨필름
6,20185361,두번할까요,"LOVE, AGAIN",2019,20191017,장편,개봉,한국,"코미디,멜로/로맨스",한국,코미디,박용집,20122736.0,(주)영화사울림
7,20190280,날씨의 아이,Weathering With You,2019,20191030,장편,개봉예정,일본,"애니메이션,판타지,멜로/로맨스",일본,애니메이션,신카이 마코토,NaN,NaN
8,20198961,더 킹: 헨리 5세,The King,2019,20191023,장편,개봉,네덜란드,드라마,네덜란드,드라마,데이비드 미쇼,NaN,NaN
9,20195005,경계선,Border,2018,20191024,장편,개봉,"스웨덴,덴마크","판타지,멜로/로맨스",스웨덴,판타지,알리 아바시,NaN,NaN


In [14]:
mlist.movieCd

0    20191029
1    20191589
2    20199950
3    20195920
4    20196136
5    20184483
6    20185361
7    20190280
8    20198961
9    20195005
Name: movieCd, dtype: int64

In [15]:
def load_movie_details(movie_list, save_path):
    
    mcodes = movie_list.movieCd
    cols = ["movieCd", "movieNm", "showTm", "nations", "genres", "actors", "companys", "audits"]    
    
    movie_details = pd.DataFrame()
    for code in mcodes:
        print(code)
        url = f"http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=430156241533f1d058c603178cc3ca0e&movieCd={code}"
        res = requests.get(url)
        minfos = res.json()["movieInfoResult"]["movieInfo"]
        
        infos = []
        for col in cols:
            infos.append(minfos[col])
        
        movieCd, movieNm, showTm = infos[:3]
        
        temp = []
        for i in range(3, len(infos)):
            tmp1 = []
            for j in range(len(infos[i])):
                tmp2 = []
                for k in range(len(infos[i][0])):
                    tmp2.append(list(infos[i][j].values())[k])
                tmp1.extend(tmp2)
            temp.append(tmp1)
        
        nations = temp[0][0]
        genre = ','.join(temp[1])
        
        actors = []
        actors_en = []
        cast = []
        cast_en = []
        for i in range(0, len(temp[2]), 4):
            if temp[2][i] == "":
                continue
            actors.append(temp[2][i])
            actors_en.append(temp[2][i+1])
        #     if temp[2][i+2] == "":
        #         continue
            cast.append(temp[2][i+2])
            cast_en.append(temp[2][i+3])
        
        companyCd = []
        companyNm = []
        companyNmEn = []
        companyPartNm = []
        for i in range(0, len(temp[3]), 4):
            companyCd.append(temp[3][i])
            companyNm.append(temp[3][i+1])
            companyNmEn.append(temp[3][i+2])
            companyPartNm.append(temp[3][i+3])
            
        watchGradeNm = []
        for i in range(0, len(temp[4]), 2):
            if temp[4][i+1] == "":
                continue    
            watchGradeNm.append(temp[4][i+1])
        
        detail = pd.DataFrame([movieCd, movieNm, showTm, nations, genre, actors, actors_en, cast, cast_en, companyCd, companyNm, companyNmEn, companyPartNm, watchGradeNm],
             index=['movieCd', 'movieNm', 'showTm', 'nations', 'genre', 'actors', 'actors_en', 'cast', 'cast_en', 'companyCd', 'companyNm', 'companyNmEn', 'companyPartNm', 'watchGradeNm']).T
        
        movie_details = pd.concat([movie_details, detail], axis=0)
    
    return movie_details

In [16]:
movie_details = load_movie_details(mlist, save_path)

20191029
20191589
20199950
20195920
20196136
20184483
20185361
20190280
20198961
20195005


In [17]:
movie_details

,movieCd,movieNm,showTm,nations,genre,actors,actors_en,cast,cast_en,companyCd,companyNm,companyNmEn,companyPartNm,watchGradeNm
0,20191029,82년생 김지영,118,한국,드라마,"[정유미, 공유, 김미경, 공민정, 박성연, 이봉련, 김성철, 이얼, 전국향, 박세...","[JUNG Yu-mi, GONG Yoo, KIM Mi-kyeong, GONG Min...","[김지영, 정대현, 미숙, 은영, 김팀장, 혜수, 지석, 영수, 큰고모, 고등학생 ...","[KIM Ji-young, JUNG Dae-hyeon, OH Mi-sook, , ,...","[20192181, 20188021, 20188021, 20188021]","[(주)봄바람영화사, 롯데컬처웍스(주)롯데엔터테인먼트, 롯데컬처웍스(주)롯데엔터테인...","[, Lotte Cultureworks Lotte Entertainment Co.,...","[제작사, 배급사, 제공, 해외세일즈사]",[12세이상관람가]
0,20191589,말레피센트 2,118,미국,"어드벤처,가족,판타지","[안젤리나 졸리, 엘르 패닝, 미셀 파이퍼]","[Angelina Jolie, Elle Fanning, Michelle Pfeiffer]","[, , ]","[, , ]","[20161801, 20161801]","[월트디즈니컴퍼니코리아 유한책임회사, 월트디즈니컴퍼니코리아 유한책임회사]","[The Walt Disney Company Korea, The Walt Disne...","[배급사, 수입사]",[12세이상관람가]
0,20199950,조커,122,미국,"액션,드라마,스릴러","[호아킨 피닉스, 재지 비츠, 로버트 드 니로, 프란시스 콘로이, 브렛 컬렌]","[Joaquin Phoenix, Zazie Beetz, Robert DeNiro, ...","[, , , , ]","[, , , , ]","[20100043, 20100043]","[워너브러더스 코리아(주), 워너브러더스 코리아(주)]","[Warner Bros. Korea, Warner Bros. Korea]","[배급사, 수입사]",[15세이상관람가]
0,20195920,가장 보통의 연애,109,한국,"멜로/로맨스,코미디","[김래원, 공효진, 강기영, 정웅인, 장소연, 주민경, 오동민, 이채은, 손영순, ...","[KIM Rae-won, KONG Hyo-jin, KANG Ki-young, JUN...","[재훈, 선영, 병철, 관수, 미영, , 현성 , 경은, 옥수수 할머니, 선영 오피...","[, , , , , , , , , ]","[20100450, 20100932, 20100932, 20142968]","[영화사 집, (주)넥스트엔터테인먼트월드(NEW), (주)넥스트엔터테인먼트월드(NE...","[ZIP CINEMA, Next Entertainment World, Next En...","[제작사, 배급사, 제공, 해외세일즈사]",[15세이상관람가]
0,20196136,람보 : 라스트 워,100,미국,액션,"[실베스터 스탤론, 파즈 베가, 세르지오 페리스-멘체타]","[Sylvester Stallone, Paz Vega, Sergio Peris-Me...","[, , ]","[, , ]","[20139196, 20161101, 20142369]","[TCO(주)더콘텐츠온, (주)제이앤씨미디어그룹, (유)조이앤시네마]","[THE CONTENTS ON, , JOY N CINEMA]","[배급사, 배급사, 수입사]",[청소년관람불가]
0,20184483,퍼펙트맨,116,한국,"코미디,드라마","[설경구, 조진웅, 허준호, 진선규, 지승현, 김사랑, 최지안]","[SUL Kyung-gu, CHO Jin-woong, HUH Joon-ho, JIN...","[장수, 영기, , , , , ]","[, , , , , , ]","[20187561, 20100103, 20100103, 20100103, 20100...","[(주)맨필름, (주)쇼박스, (주)쇼박스, (주)쇼박스, (주)쇼박스]","[, Showbox Corp, Showbox Corp, Showbox Corp, S...","[제작사, 제작사, 배급사, 제공, 해외세일즈사]",[15세이상관람가]
0,20185361,두번할까요,112,한국,"코미디,멜로/로맨스","[권상우, 이정현, 이종혁, 성동일, 정상훈, 김현숙, 박경혜, 구성환]","[KWON Sang-woo, LEE Jung-hyun, LEE Jong-hyuk, ...","[현우, 선영, 상철, , , , 김 간호사 , 차 대리 ]","[, , , , , , , ]","[20122736, 20139617, 20111211, 20139617, 20142...","[(주)영화사울림, (주)리틀빅픽쳐스, 케이티하이텔(주), (주)리틀빅픽쳐스, (주...","[Woollim, Little Big Pictures, kth, Little Big...","[제작사, 배급사, 제공, 제공, 해외세일즈사]",[15세이상관람가]
0,20190280,날씨의 아이,112,일본,"애니메이션,판타지,멜로/로맨스","[다이고 코타로, 모리 나나, 오구리 슌]","[Kotaro Daigo, Nana Mori, Shun Oguri]","[, , ]","[, , ]","[20122956, 20191941, 20122956]","[(주)미디어캐슬, 워터홀컴퍼니(주), (주)미디어캐슬]","[, WATERHOLE COMPANY, ]","[배급사, 배급사, 수입사]",[15세이상관람가]
0,20198961,더 킹: 헨리 5세,140,네덜란드,드라마,"[티모시 샬라메, 조엘 에저튼, 숀 해리스]","[Timothee Chalamet, Joel Edgerton, Sean Harris]","[, , ]","[, , ]",[20100285],[판씨네마(주)],[Pancinema Corp.],[배급사],[청소년관람불가]
0,20195005,경계선,110,스웨덴,"판타지,멜로/로맨스","[에바 멜란데르, 에로 밀로노프]","[Eva Melander, Eero Milonoff]","[, ]","[, ]","[20161101, 20142369]","[(주)제이앤씨미디어그룹, (유)조이앤시네마]","[, JOY N CINEMA]","[배급사, 수입사]",[청소년관람불가]


In [18]:
movie_details.to_csv(os.path.join(save_path, "movie_hot_details.csv"), index=None, encoding="euc-kr")

In [78]:
mcodes = mlist.movieCd.to_list()

In [79]:
code = mcodes[2]

In [80]:
code

20129370

In [81]:
url = f"http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=430156241533f1d058c603178cc3ca0e&movieCd={code}"

In [82]:
res = requests.get(url)
minfos = res.json()["movieInfoResult"]["movieInfo"]

In [87]:
cols = ["movieCd", "movieNm", "showTm", "nations", "genres", "actors", "companys", "audits"]

In [88]:
infos = []
for col in cols:
    infos.append(minfos[col])

In [113]:
movieCd, movie_Nm, showTm = infos[:3]

In [95]:
temp = []
for i in range(3, len(infos)):
    tmp1 = []
    for j in range(len(infos[i])):
        tmp2 = []
        for k in range(len(infos[i][0])):
            tmp2.append(list(infos[i][j].values())[k])
        tmp1.extend(tmp2)
    temp.append(tmp1)

In [96]:
nations = temp[0][0]

In [97]:
genre = ','.join(temp[1])

In [98]:
actors = []
actors_en = []
cast = []
cast_en = []
for i in range(0, len(temp[2]), 4):
    if temp[2][i] == "":
        continue
    actors.append(temp[2][i])
    actors_en.append(temp[2][i+1])
#     if temp[2][i+2] == "":
#         continue
    cast.append(temp[2][i+2])
    cast_en.append(temp[2][i+3])

In [99]:
companyCd = []
companyNm = []
companyNmEn = []
companyPartNm = []
for i in range(0, len(temp[3]), 4):
    companyCd.append(temp[3][i])
    companyNm.append(temp[3][i+1])
    companyNmEn.append(temp[3][i+2])
    companyPartNm.append(temp[3][i+3])

In [100]:
watchGradeNm = []
for i in range(0, len(temp[4]), 2):
    if temp[4][i+1] == "":
        continue    
    watchGradeNm.append(temp[4][i+1])

In [101]:
detail = pd.DataFrame([movieCd, movie_Nm, showTm, nations, genre, actors, actors_en, cast, cast_en, companyCd, companyNm, companyNmEn, companyPartNm, watchGradeNm],
             index=['movieCd', "movie_Nm", 'showTm', 'nations', 'genre', 'actors', 'actors_en', 'cast', 'cast_en', 'companyCd', 'companyNm', 'companyNmEn', 'companyPartNm', 'watchGradeNm']).T

In [102]:
detail

,movieCd,movie_Nm,showTm,nations,genre,actors,actors_en,cast,cast_en,companyCd,companyNm,companyNmEn,companyPartNm,watchGradeNm
0,20129370,명량,128,한국,"사극,액션","[최민식, 류승룡, 조진웅, 김명곤, 진구, 이정현, 권율, 노민우, 김태훈, 이승...","[CHOI Min-shik, RYU Seung-ryong, CHO Jin-woong...","[이순신 , 구루지마 , 와키자카 , 도도, 임준영 , 정씨 여인 , 이회, 하루,...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[20123063, 20110854, 20110854, 20100109]","[(주)빅스톤픽쳐스, 씨제이이앤엠(주), 씨제이이앤엠(주), 씨제이엔터테인먼트]","[BIGSTONEPICTURES, CJ ENM Corp., CJ ENM Corp.,...","[제작사, 배급사, 제공, 해외세일즈사]",[15세이상관람가]


In [ ]:
temp2 = []
for i in range(2, len(infos)):
    tmp2 = []
    for j in range(len(infos[i])):
        tmp3 = []
        for k in range(len(infos[i][0])):
            if list(infos[i][j].values())[k] == "":
                continue
            tmp3.append(list(infos[i][j].values())[k])
        tmp2.append(tmp3)
    temp2.append(tmp2)
temp2

In [228]:
pd.DataFrame(tmp1).T

,0,1,2,3,4,5
0,[미국],[SF],"[샘 워싱턴, Sam Worthington]","[필름, 필름]","[20101507, 이십세기폭스필름코퍼레이션, 20th Century Fox Fil...","[2009-F430, 12세이상관람가]"
1,None,[액션],"[조 샐다나, Zoe Saldana]","[2D, 디지털]","[20175001, 주식회사 해리슨앤컴퍼니, Harrison & Company, 배급사]","[2018-MF00904, 12세이상관람가]"
2,None,[어드벤처],"[시고니 위버, Sigourney Weaver]","[3D, 3D 디지털]","[20100041, 이십세기폭스코리아(주), 20th Century Fox Kore...",None
3,None,None,None,"[4D, 4D]","[20175001, 주식회사 해리슨앤컴퍼니, Harrison & Company, 수입사]",None
4,None,None,None,"[IMAX, IMAX]","[20100041, 이십세기폭스코리아(주), 20th Century Fox Kore...",None
